In [ ]:
import numpy as np
import pandas as pd
import sys
import os
try:
    import openmm as mm
    import openmm.app as app
    import openmm.unit as unit
except ImportError:
    import simtk.openmm as mm
    import simtk.openmm.app as app
    import simtk.unit as unit
import mdtraj
try:
    import nglview
except ImportError:
    print('Please install nglview to visualize molecules in the jupyter notebooks.')

sys.path.append('../..')

from openabc.forcefields.parsers import SMOGParser, DNA3SPN2Parser
from openabc.forcefields import SMOG3SPN2Model
from openabc.utils.helper_functions import get_WC_paired_sequence
from openabc.utils.insert import insert_molecules
# import some functions useful for setting up chromatin related simulations
from openabc.utils.chromatin_helper_functions import get_chromatin_rigid_bodies


In this tutorial, we will show how to use SMOG and 3SPN2 models to study nucleosomes. Nucleosome is a protein-DNA complex. It is composed of a protein octamer called histone, and 147 bp DNA wrapped around the histone. It is the basic unit of genome organization, and it is appropriate to use these near-atomistic CG models to study nucleosomes. 

Our group has used the SMOG + 3SPN2 model to understand nucleosome and chromatin properties, as shown in the following references:

(1) Ding, Xinqiang, Xingcheng Lin, and Bin Zhang. "Stability and folding pathways of tetra-nucleosome from six-dimensional free energy surface." Nature communications 12.1 (2021): 1091.

(2) Lin, Xingcheng, et al. "Cooperative DNA looping by PRC2 complexes." Nucleic Acids Research 49.11 (2021): 6238-6248.

(3) Liu, Shuming, Xingcheng Lin, and Bin Zhang. "Chromatin fiber breaks into clutches under tension and crowding." Nucleic Acids Research 50.17 (2022): 9738-9747. 

All the SMOG+3SPN2 simulations were performed with LAMMPS in the 3 references. Use the third reference as an example, we try to keep our settings consistent with the simulations in the third reference. 

Let's begin with loading the structures of a single nucleosome. The protein and DNA structures are both from PDB 1KX5. 

So when we simulate nucleosomes, we keep using rigid body settings. By default, we rigidize the histone core and the middle 73 bp wrapped around histone as a rigid body. Such rigid body settings help stabilize the stable histone core with inner layer DNA and facilitate a large timestep. Meanwhile, we remove all the native pairs and dihedrals that involve any histone tail atoms, since histone tails are disordered. By considering these two effects, (1) we do not need any native pairs or dihedrals (because any histone atom is in either histone core or histone tail); (2) we want to set rigid body for histone core and middle 73 bps. Notably, even if there is no dihedral potential for histones, nonbonded interactions between 1-4 atom pairs in histone tails are still excluded (as in LAMMPS simulations we keep doing this, though this should not affect the results too much). 

Since 3SPN2 model can capture DNA sequence, we need to load the DNA with sequence information. In the case of nucleosome, the CG DNA configuration is converted from atomistic DNA in 1KX5, but the sequence is reset with the one saved in dna_seq.txt. So we can easily manipulate the sequence of the CG DNA.  

Important notes:

To run 3SPN2 force field with B_curved DNA, **X3DNA** is required. Please install X3DNA and set the environment variable as 

```
export X3DNA='path/to/x3dna'
``` 

In [ ]:
n_nucl = 2 # the number of nucleosomes
platform_name = 'CUDA' # such simulations are slow on single CPU, so we expect the user to use GPU

# load single nucleosome
single_nucl = SMOG3SPN2Model()

# load histone
histone = SMOGParser.from_atomistic_pdb('single-nucl-pdb-files/histone.pdb', 'histone_CA.pdb',
                                        default_parse=False)
histone.parse_mol(get_native_pairs=False) # no need to get native pairs
single_nucl.append_mol(histone)

# load DNA with target sequence shown in dna_seq.txt
# target_seq is the full sequence, as 3SPN2 model does not enforce W-C paired sequence
with open('dna_seq.txt', 'r') as f:
    seq1 = f.readlines()[0].strip()
seq2 = get_WC_paired_sequence(seq1)
target_seq = seq1 + seq2
dna = DNA3SPN2Parser.from_atomistic_pdb('single-nucl-pdb-files/dna.pdb', 'cg_dna.pdb', new_sequence=target_seq)
single_nucl.append_mol(dna)

# write pdb of single nucleosome
# we need this pdb as we will construct the initial configuration for two-nucleosome system
single_nucl.atoms_to_pdb('cg_nucl.pdb')

Now let's prepare the system of two nucleosomes. 

In [ ]:
# prepare the system composed of two nucleosomes
two_nucl = SMOG3SPN2Model()
box_a, box_b, box_c = 50, 50, 50
for i in range(n_nucl):
    two_nucl.append_mol(histone)
    two_nucl.append_mol(dna)
insert_molecules('cg_nucl.pdb', 'two_cg_nucl.pdb', n_mol=n_nucl, box=[box_a, box_b, box_c])

Now add forces and run the simulation. Note the rigid body is realized by adding restraints, so we need to to set a rigid body configuration as reference. For simplicity, just use the inital configuration as the rigid body configuration. 

In [ ]:
top = app.PDBFile('two_cg_nucl.pdb').getTopology()
init_coord = app.PDBFile('two_cg_nucl.pdb').getPositions()
rigid_coord = init_coord
two_nucl.create_system(top, box_a=box_a, box_b=box_b, box_c=box_c)
# set rigid bodies and remove bonded interactions within the same rigid body
# the two rigid bodies are saved in rigid_bodies
# set rigid body with set_rigid_bodies method after creating the system and before adding forces
single_nucl_rigid_body = get_chromatin_rigid_bodies(n_nucl=1, nrl=147, n_rigid_bp_per_nucl=73)[0]
n_atoms_per_nucl = len(single_nucl.atoms.index)
rigid_bodies = [single_nucl_rigid_body, (np.array(single_nucl_rigid_body) + n_atoms_per_nucl).tolist()]
two_nucl.set_rigid_bodies(rigid_coord, rigid_bodies)
two_nucl.add_protein_bonds(force_group=1)
two_nucl.add_protein_angles(force_group=2)
# do not add histone dihedrals, as the dihedrals either involve histone tail atoms, or is within histone core
two_nucl.add_native_pairs(force_group=4)
two_nucl.add_dna_bonds(force_group=5)
two_nucl.add_dna_angles(force_group=6)
two_nucl.add_dna_stackings(force_group=7)
two_nucl.add_dna_dihedrals(force_group=8)
two_nucl.add_dna_base_pairs(force_group=9)
two_nucl.add_dna_cross_stackings(force_group=10)
two_nucl.parse_all_exclusions() # note nonbonded interactions between 1-4 atom pairs in histone tails are excluded
two_nucl.add_all_vdwl(force_group=11)
two_nucl.add_all_elec(force_group=12)

temperature = 300*unit.kelvin
friction_coeff = 0.01/unit.picosecond
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
two_nucl.set_simulation(integrator, platform_name=platform_name, init_coord=init_coord)
two_nucl.simulation.minimizeEnergy()
two_nucl.add_reporters(report_interval=100, output_dcd='output.dcd')
two_nucl.simulation.context.setVelocitiesToTemperature(temperature)
two_nucl.simulation.step(500)

In [ ]:
# view trajectory
traj = mdtraj.load_dcd('output.dcd', top='two_cg_nucl.pdb')
traj.xyz -= np.mean(traj.xyz, axis=1, keepdims=True) # realign to the origin
view = nglview.show_mdtraj(traj)
view